In [1]:
import pandas as pd

files = r'_SNP_dataset\financials.parquet'
col = ['companyId', 'companyName', 'periodTypeName', 'periodEndDate', 'fiscalQuarter', 'fiscalYear',
       'filingDate', 'instanceDate', 'ISOCode', 'restatementTypeName', 'latestPeriodFlag', 'dataItemId',
       'dataItemName', 'dataItemValue', 'unitTypeName', 'unitTypeValue', 'nmFlag', 'fiscalChainSeriesId']

# 데이터프레임을 6분할하기 위한 컬럼 리스트
col_list = [col[i:i + 3] for i in range(0, len(col), 3)]

# 각 분기별 데이터를 분할하고 저장
for quarter in range(1, 5):
    for i, cols in enumerate(col_list):
        # 해당 열 그룹만 읽기
        df = pd.read_parquet(files, engine='fastparquet', columns=cols)
        quarter_size = len(df) // 4
        start_row = (quarter - 1) * quarter_size
        end_row = quarter * quarter_size if quarter < 4 else len(df)
        df_quarter = df.iloc[start_row:end_row]
        
        # 분할된 데이터프레임 저장
        df_quarter.to_parquet(f'./financials_col{i+1}_quarter{quarter}.parquet')
        del df, df_quarter

# 분할된 데이터프레임 결합 및 저장
for quarter in range(1, 5):
    df_list = [pd.read_parquet(f'./financials_col{i+1}_quarter{quarter}.parquet') for i in range(6)]
    combined_df = pd.concat(df_list, axis=1)
    combined_df.to_parquet(f'financials_{quarter}.parquet')
    del combined_df


In [2]:
df = pd.read_parquet(f'./financials_{1}.parquet', engine='fastparquet')
df

,companyId,companyName,periodTypeName,periodEndDate,fiscalQuarter,fiscalYear,filingDate,instanceDate,ISOCode,restatementTypeName,latestPeriodFlag,dataItemId,dataItemName,dataItemValue,unitTypeName,unitTypeValue,nmFlag,fiscalChainSeriesId
0,694525289.0,"RYUKYU ASTEEDA Sports Club Co., Ltd.",Quarterly,2021-03-31,1.0,2021.0,2021-08-13,2021-09-02 20:08:00,JPY,Press Release,0.0,2018.0,Change In Accounts Receivable,18.500,Millions,1000000.0,0.0,1.0
1,694525289.0,"RYUKYU ASTEEDA Sports Club Co., Ltd.",Quarterly,2021-03-31,1.0,2021.0,2021-08-13,2021-09-02 20:08:00,JPY,Press Release,0.0,2047.0,"Other Operating Activities, Total",7.500,Millions,1000000.0,0.0,1.0
2,694525289.0,"RYUKYU ASTEEDA Sports Club Co., Ltd.",Quarterly,2021-03-31,1.0,2021.0,2021-08-13,2021-09-02 20:08:00,JPY,Press Release,0.0,2005.0,Cash from Investing,-11.500,Millions,1000000.0,0.0,1.0
3,694525289.0,"RYUKYU ASTEEDA Sports Club Co., Ltd.",Quarterly,2021-03-31,1.0,2021.0,2021-08-13,2021-09-02 20:08:00,JPY,Press Release,0.0,2150.0,Net Income - (CF),-38.000,Millions,1000000.0,0.0,1.0
4,694525289.0,"RYUKYU ASTEEDA Sports Club Co., Ltd.",Quarterly,2021-03-31,1.0,2021.0,2021-08-13,2021-09-02 20:08:00,JPY,Press Release,0.0,2045.0,Change in Other Net Operating Assets,-22.000,Millions,1000000.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73855758,254111553.0,"Rightside Group, Ltd.",Quarterly,2017-03-31,1.0,2017.0,2017-05-09,2017-05-10 06:57:00,USD,Original,1.0,2036.0,"Long-Term Debt Repaid, Total",-13.783,Millions,1000000.0,0.0,1.0
73855759,254111553.0,"Rightside Group, Ltd.",Quarterly,2017-03-31,1.0,2017.0,2017-05-09,2017-05-10 06:57:00,USD,Original,1.0,2081.0,Net Cash From Discontinued Operations,-2.300,Millions,1000000.0,0.0,1.0
73855760,254111553.0,"Rightside Group, Ltd.",Quarterly,2017-03-31,1.0,2017.0,2017-05-09,2017-05-10 06:57:00,USD,Original,1.0,2004.0,Cash from Financing,-15.697,Millions,1000000.0,0.0,1.0
73855761,254111553.0,"Rightside Group, Ltd.",Quarterly,2017-03-31,1.0,2017.0,2017-05-09,2017-05-10 06:57:00,USD,Original,1.0,2169.0,Issuance of Common Stock,0.031,Millions,1000000.0,0.0,1.0


In [13]:
quarter = [1, 2, 3, 4]

for q in quarter:
    df = pd.read_parquet(f'./_SNP_dataset/market_combined_quarter_{q}.parquet', engine='fastparquet')
    # exchangeSymbol로 구분해서 exchangeSymbol별로 parquet 저장
    exchangeSymbol = df['exchangeSymbol'].unique()
    for i in exchangeSymbol:
        df_ex = df[df['exchangeSymbol'] == i]
        df_ex.to_parquet(f'./_SNP_dataset/exchange/market_quarter_{q}_{i}.parquet')